In [240]:
import numpy as np
import numpy.matlib 
import math
import matplotlib.pyplot as plt
import scipy.stats

# Inspired by youtube channel D&D Optimised (now d4: D&D Deep Dive)

## Calculate average damage

### Source: D&D Optimised
https://www.youtube.com/watch?v=nk2SIaGXSYo

"The basic formula for calculating average damage is as follows:				
((Average Weapon Damage + Applicable per-hit Damage Modifiers)*(% chance to hit))+((Average Dice Damage on attack)*(% chance to crit))+any once per round damage													
													
Keep in mind that if you have multiple attacks, you need to run the above for each attack - you can simply multiply everything between the outermost parentheses by the # of attacks *if* those attacks are identical,													
but in the case of someone using a Polearm Master Feat, for example, they are not, as the regular attacks would use a D10 for the weapon, wheras the bonus action attack would use a D4 for the weapon."													

### Source: https://www.dndbeyond.com/sources/phb/combat#MakinganAttack
"To make an attack roll, roll a d20 and add the appropriate modifiers. If the total of the roll plus modifiers equals or exceeds the target's Armor Class (AC), the attack hits." 


## Detailed calculations

### Definitions
[roll]: Average Dice Roll
OAC: Opponents Armour Class

### Chance to hit and chance to miss

P(My hit was greater or equal than the Opponents AC) = P([roll]+ hit bonus >= OAC) => P([roll]>= OAC- hit bonus)

P(hit)=P([rol] >= OAC -hit bonus) = [ 20 -(OAC- hit bonus)+1]/20 = 1 -(OAC- hit bonus -1)/20 = 1 - P(miss)

P(miss) = (OAC- hit bonus -1)/20

### Chance to hit with advantage
P (hit on first roll but not on the second)+ P (hit on the second roll but not on the first)+ P(hit on both rolls)=  2  * [ 1 -(OAC- hit bonus -1)/20] * [ (OAC- hit bonus -1)/20]+ [ 1 -(OAC- hit bonus -1)/20] **2 (see also binomial distribution).

The shorter way is to calculate "1 - the chance of missing both rolls". 

P (miss both rolls) = [P(miss)] = [(OAC- hit bonus -1)/20]**2

P( hit with an advantage) = 1 - [(OAC- hit bonus -1)/20]**2

### Chance to hit with advantage and Elven Accuracy (or Lucky)
Elven accuracy, i.e roll one more time: 1- [(OAC- hit bonus -1)/20]**3

### Critical hit  (minimum chance to hit)
Crit hit (natural 20): p(critit)= 1/20

### Critical hit with advantage
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**2

### Minimum chance to hit with advantage and Elven Accuracy (or Lucky)
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**3

### Minimum chance to hit with Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)

In [241]:
1- (1-2/20)

0.09999999999999998

### Minimum chance to hit with advantage, Elven Accuracy (or Lucky) and Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)**3

In [242]:
1- (1-2/20)**3

0.2709999999999999

One way to obtain advantage as a Warlock is to get Devil's Sight and Cast Darkness on oneself (when multiclass with e.g. fighter, one may use the blind fighting style instead of Devil's Sight.

##  Calculating the Probablility of critical hits (rounded to 5 decimal digits).

In [243]:
import dnd_optimisation as dndopt # this is a python file containing the calcuations for critical hits

In [244]:
import importlib  #This is because I am making changes to the file all the time ;) 
importlib.reload(dndopt)  

<module 'dnd_optimisation' from '/Users/eleni/Python Scripts/DnD/dnd_optimisation.py'>

#### Setting advantage, luck_point (related to the lucky feat) and elven_accuracy to false gives the probabity of a natural critical hit (natural 20) 1/20

In [245]:
advantage=False; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [246]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.05

#### Only advantage

In [247]:
advantage=True; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [248]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.0975

#### Advantage and lucky

In [249]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=False

In [250]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.14263

#### Hexblade curse with advantage

In [251]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=True

In [252]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.271

# Comparing different builds

## Warlock (Celestial Patron) with 2 levels of Fighter vs 2 levels of Paladin for maximal damage 

### Assuming the following Ability Scores

In [253]:
AbilityScores= numpy.array([19, 14, 12, 16, 8, 18])

In [254]:
Abilities=['STR', 'DEX', 'CON','INT','WIS', 'CHA']

### Calculate the Ability Score Modifiers

In [255]:
Modifiers = np.floor((AbilityScores - 10)/2)

In [256]:
Modifiers=Modifiers.astype(int)

### What is the Ability Score for Charisma?

In [257]:
AbilityScores[Abilities.index('CHA')]

18

### What is the Modifier for Charisma?

In [258]:
Modifiers[Abilities.index('CHA')]

4

### Define the Character's Level

In [259]:
Level=8;

### Calculate the Proficiency Bonus

In [260]:
ProficiencyBonus=1+np.ceil(Level/4)

In [261]:
ProficiencyBonus=ProficiencyBonus.astype(int)

### Calculate Hit for Spells Attacks casted with Charisma

In [262]:
OtherBonus=1; #Rod of pact keeper

In [263]:
SpellHit=Modifiers[Abilities.index('CHA')]+ProficiencyBonus+OtherBonus

In [264]:
SpellHit

8

### Calculate Hit for Weapon Attacks with Strength

In [265]:
WeaponBonus=1; #Magical +1 weapon

In [266]:
MeleeHit=Modifiers[Abilities.index('STR')]+ProficiencyBonus+WeaponBonus

In [267]:
MeleeHit

8

In [268]:
### Calculate Hit for Weapon Attacks with Dexterity

In [269]:
DexMeleeHit=Modifiers[Abilities.index('DEX')]+ProficiencyBonus+WeaponBonus

In [270]:
DexMeleeHit

6

### The Opponent's Armor Class is:

In [271]:
OAC=15

## Calculate the damage of Eldrich Blast

### Probability to hit an Opponent

#### Probability to hit the Opponent (for 1 dice roll)

In [272]:
PrHit= 1 -(OAC- SpellHit -1)/20 

In [273]:
PrHit

0.7

#### using a function to calculate the probability 
####  probability_of__hitting_opponent(opponent_AC,modifiers,advantage,luck_point,elven_accuracy)

In [274]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,0,0,0)

0.7

#### Probability to miss the Opponent (for 1 dice roll)

In [275]:
PrMiss=(OAC- SpellHit -1)/20 

In [276]:
PrMiss

0.3

#### Probability to hit the opponent with advantage (Ad=2)

In [277]:
Ad=2

In [278]:
PrHitA=1-(PrMiss)**(Ad)

In [279]:
PrHitA

0.91

In [280]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,1,0,0)

0.91

#### Number of Elrdich Blast Rays

In [281]:
Rays=min(math.floor(Level/5)+1,4)

In [282]:
Rays

2

#### In that case the average damage (per ray) will be (with Agonizing Blast) 1d10+ Charisma Modifier

In [283]:
AverageAgonisingBlastDamage=5

In [284]:
AverageDamagePerRay=AverageAgonisingBlastDamage+Modifiers[Abilities.index('CHA')]

In [285]:
AverageDamagePerRay

9

Additional modifier for Hex spell

In [286]:
HexAverageDamage=3

### Average damage, taking into account critical hits, parametrised by the number of rays

For two rays only the formula would be 

 2 x Probability that only one ray will hit x damage + Probability that both rays will hit x 2 x damage + 2 x Probability that only one dice will crit hit x damage  + Probability that both dices will hit x 2 x damage 

In [287]:
Critit=1/20;

In [288]:
2 * PrHit * PrMiss * AverageDamagePerRay+ 2* (Critit) *(1-Critit) * AverageDamagePerRay+ PrHit**2 * 2* AverageDamagePerRay + 2* (Critit) *(Critit) * AverageDamagePerRay

13.499999999999998

In order to easily generalise to higher number of rays we use the binomial distribution. we check for 2 Rays to see the same result as above.

Without taking into account Crit Hit

In [289]:
ADNCR = scipy.stats.binom.pmf(1,2,PrHit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,PrHit)* 2* AverageDamagePerRay 

In [290]:
ADNCR

12.6

Calculating Crit Hit

In [291]:
ACHD= scipy.stats.binom.pmf(1,2,Critit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,Critit)* 2* AverageDamagePerRay 

In [292]:
ACHD

0.9000000000000002

Adding the two together

In [293]:
TAD=ADNCR+ACHD

In [294]:
TAD

13.5

#### Calculating for arbitrary number of Rays

In [295]:
TotalDamage=0;

In [296]:
indexRay = 1
while indexRay < (Rays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,Rays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,Rays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [297]:
TotalDamage

13.5

### Average damage with  Advantage

In [338]:
n=1

In [339]:
TotalRays=n*Rays

In [340]:
CrititA=1-(1-1/20)**2

In [341]:
TotalDamage=0

In [342]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [343]:
TotalDamage

18.135

### Average damage with Advantage and Hex

In [314]:
n=1

In [315]:
TotalRays=n*Rays

In [316]:
TotalDamage=0

In [317]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [318]:
TotalDamage

24.180000000000003

### Average damage with  n repetitions (e.g. Action Surge n=2)

In [319]:
n=2;

In [320]:
TotalRays=n*Rays

In [321]:
TotalDamage=0

In [322]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [323]:
TotalDamage

27.000000000000007

### Average damage with  n repetitions  (e.g. Action Surge n=2) and Advantage

For rolls with advantage I need to use in the formula the probability of hitting the target with advantage

In [324]:
n=2;

In [325]:
TotalRays=n*Rays

In [326]:
TotalDamage=0

In [327]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [328]:
  TotalDamage

36.27000000000001

### Average damage with  n repetitions  (e.g. Action Surge n=2), Advantage and Hex

In [329]:
n=2;

In [330]:
TotalRays=n*Rays

In [331]:
TotalDamage=0

In [332]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [333]:
TotalDamage

48.36

#### Using the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [346]:
repetitions=4; average_damage_per_hit=AverageDamagePerRay+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [347]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

48.36

## Calculate the damage of Melee + Hex + Smite with Advantage

In [79]:
WeaponDamage=9 #Longsword +1

In [80]:
PrHit= 1 -(OAC- MeleeHit -1)/20 

In [81]:
PrHit

0.7

In [82]:
PrMiss=1-PrHit

In [83]:
Ad=2

In [84]:
PrHitAd=1-(PrMiss)**(Ad)

In [85]:
PrHitAd

0.9099999999999999

In [86]:
Smite=20

In [87]:
BonusSmite=6

In [88]:
TotalDamage= (WeaponDamage+Smite+BonusSmite+HexAverageDamage)*PrHitAd+ (WeaponDamage+Smite+BonusSmite+HexAverageDamage) *CrititA

In [89]:
TotalDamage

38.285